In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

In [ ]:
scenario = "2025-07-08_4"
filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "case_study_eurostar", scenario, "eurostar.csv")
df = pd.read_csv(filename)
print(df.columns)
trains = df['trainNumber'].unique()
id_to_train_num = {}
for id in df['id'].unique():
    id_to_train_num[int(id)] = int(df[df['id'] == id]['trainNumber'].unique()[0])
for t in trains:
    df_train = df[df['trainNumber'] == t]
    for i, ut in df_train.iterrows():
        print(f"Train {t} from {df_train['origin'].unique()[0]} at {df_train['start_time'].unique()[0]} to {df_train['destination'].unique()[0]} at {df_train['endTime'].unique()[0]} has {len(df_train)} entries, {i}th entry: delay at {ut['delay_location']} for {ut['delay_amount']} with ATF{ut['zeta'], ut['alpha'], ut['beta'], ut['delta']}")
        print("Path of Eurostar", ut['path'])
df

In [ ]:
cities = {
    "Gv": "The Hague",
    "Dt": "Delft",
    "Dtcp": "Delft",
    "Laa": "The Hague",
    "Rtd": "Rotterdam",
    "Shl": "Schiphol",
    "Hfd": "Hoofddorp",
    "Ledn": "Leiden",
    "Sdm": "Schiedam",
    "Rmoa": "Rotterdam",
    "Gvm": "The Hague"
}

In [ ]:
allowed_delays = [180]

def td_str(td):
    return ':'.join(re.split(r'[:.]+', str(td)) [1:3])

def extract_tipping_point(df):
    def apply_func(df):

        result=df.groupby("Location").agg({
            "beta": "max",
            "Delay Amount": "max",
        })
        # result = result.loc[result['beta'] < 900]
        # result["Tipping Point (sec)"] = result["beta"]
        result["Tipping Point"] = result["beta"].apply(lambda x: td_str(timedelta(seconds=x)))

        # def tp_finder(x):
        #     new_tp = x['beta'] - max(0, x['Delay Amount'] - allowed_delay)
        #     if new_tp > 0:
        #         return td_str(timedelta(seconds=new_tp))
        #     return "-"

        # for allowed_delay in allowed_delays:
        #     result[f"Tipping Point ({allowed_delay}s)"] = result.apply(tp_finder, axis=1)
        # result["Delay Amount (sec)"] = result["Delay Amount"]
        result["Delay Amount"] = result["Delay Amount"].apply(lambda x: td_str(timedelta(seconds=x)))
        return result.sort_values("Tipping Point", ascending=True).drop(columns=["beta"])

    df["Delay Location"] = df["Delay Location"].str.split("|").apply(lambda x: x[0])
    df["Location"] = df["Delay Location"].apply(lambda x: cities[x] if "_" not in x else cities[x.split("_")[0]])
    df = df.groupby(by='Train ID').apply(apply_func, include_groups=False)
    return df

tp_df = df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "id": "Train ID",
    "scenario": "Scenario",
    "label": "Label",
})
tp_df["Scenario"] = tp_df["Scenario"].apply(lambda x: x.split(".")[0])
tp_df["Delay Location"] = tp_df["Delay Location"].apply(lambda x: x.split("-")[1])
tp_df = tp_df.groupby(["Scenario"]).apply(extract_tipping_point, include_groups=False)
# tp_df = tp_df.rename(columns={"Tipping Point": "Tipping Point (safe=beta)"})
tp_df

In [ ]:
lines = tp_df.to_latex().split("\n")
latex_filename = filename.replace("eurostar.csv", "table_tipping_points.tex")
new_lines = []
i = 0
while i < len(lines):
    if "\begin{tabular}" in lines[i]:
        new_lines.append(r'\begin{tabular}{llll}')
    elif "cline" in lines[i]:
        pass
    elif lines[i] == r'\toprule':
        new_lines.append(r'\toprule')
        new_lines.append(r'Train & Location & Delay & Tipping point \\')
        i += 2
    elif lines[i][0:3] == r' & ':
        # if "multirow" in lines[i]:
        #     new_lines.append(r' & '.join([lines[i].split(r' & ')[1].split("{")[-1].replace("}", "")] + lines[i].split(r' & ')[2:]))
        # else:
        new_lines.append(r' & '.join(lines[i].split(r' & ')[1:]))
    elif "2025-07-08" in lines[i]:
        new_lines.append(r' & '.join(lines[i].split(r' & ')[1:]))
    else:
        new_lines.append(lines[i])
    i += 1
# for line in new_lines:
#     print(line)
with open(latex_filename, "w") as f:
    f.write("\n".join(new_lines))

# Comparison

In [ ]:
comp_df = df.groupby(["scenario", "label"])["path"].count()

comp_df

# Runtime results

In [ ]:
scenario = "2025-07-08_4"
time_filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "case_study_eurostar", scenario, "eurostar-time.csv")
time_df = pd.read_csv(time_filename)
time_df = time_df.round(2).astype(str)
pv = time_df.pivot_table(
    columns=["scenario", "label"],
    values = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks", "track graph creation", "routing graph creation", "FlexSIPP search time"]
)
pv.to_latex(time_filename.replace("eurostar-time.csv", "time_comparison.tex"))
